Obtain Training Data

In [13]:
import pandas as pd
dataframe = pd.read_csv('./data/orders.csv')
dataframe['product_id'].unique()

array([293073, 926868, 417009, 424718, 799380, 644730, 417528, 575961,
       740474, 129553, 240196, 966126, 876627, 457848, 357966, 846554,
       542696, 203454, 656573, 855119, 741835, 391708, 120672, 221626,
       782729])

Feature Engineering

In [14]:
# Use later to estimate revenue
priceData = dataframe[['product_id', 'unit_price']]

# Convert delivery_time to datetime
dataframe['delivery_time'] = pd.to_datetime(
    dataframe['delivery_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce'
)

# Extract hour (24-hour format)
dataframe['hour'] = dataframe['delivery_time'].dt.hour
print("Hour values:", sorted(dataframe['hour'].unique()))

# Convert order_date to datetime and get day of week
dataframe['order_date'] = pd.to_datetime(dataframe['order_date'])
dataframe['day_of_week'] = dataframe['order_date'].dt.day_name()

# Drop columns to avoid confusion
dataframe.drop(['order_id', 'unit_price', 'delivery_time', 'order_date'], axis=1, inplace=True)

Hour values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [15]:
dataframe.to_csv('./data/feautredOrders.csv', index=False)

Preprocess

In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Encode day_of_week
day_encoder = LabelEncoder()
dataframe['day_of_week'] = day_encoder.fit_transform(dataframe['day_of_week'])

# Features and targets
area_encoder = LabelEncoder()
dataframe['area_encoded'] = area_encoder.fit_transform(dataframe['area'])

X = dataframe[['hour', 'day_of_week', 'area_encoded']]

product_encoder = LabelEncoder()
dataframe['product_label'] = product_encoder.fit_transform(dataframe['product_id'])  # ✅ New
y1 = dataframe['product_label']
num_classes = len(product_encoder.classes_) 
y2 = dataframe['quantity']                                   # regression
y3 = dataframe['area_encoded']

dataframe['quantity_class'] = dataframe['quantity'] - 1  # 0-indexed for softmax

# Targets
y_quantity_class = dataframe['quantity_class']

# Train-test split
X_train_full, X_test_full, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(
    X, y1, y_quantity_class, y3, test_size=0.2
)

X_train = X_train_full[['hour', 'day_of_week']].values  # Final Keras inputs
X_test = X_test_full[['hour', 'day_of_week']].values


Models

In [17]:
import lightgbm as lgb

X_class = dataframe[['hour', 'day_of_week']]
y_class = dataframe['product_label']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.2)
lgbm_clf = lgb.LGBMClassifier()
lgbm_clf.fit(Xc_train, yc_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 2058, number of used features: 2
[LightGBM] [Info] Start training from score -3.235041
[LightGBM] [Info] Start training from score -3.298757
[LightGBM] [Info] Start training from score -3.235041
[LightGBM] [Info] Start training from score -3.175143
[LightGBM] [Info] Start training from score -3.285684
[LightGBM] [Info] Start training from score -3.175143
[LightGBM] [Info] Start training from score -3.175143
[LightGBM] [Info] Start training from score -3.298757
[LightGBM] [Info] Start training from score -3.014369
[LightGBM] [Info] Start training from score -3.285684
[LightGBM] [Info] Start training from score -3.152153
[LightGBM] [Info] Start training from score -3.034370
[LightGBM] [Info] Start training from score -3.175143
[LightGBM] [I

LGBMClassifier()

In [18]:
import tensorflow as tf
import numpy as np
from sklearn.utils.class_weight import compute_sample_weight

# Generate sample weights for 'quantity'
sample_weights_quantity = compute_sample_weight(
    class_weight='balanced',
    y=y_quantity_class.iloc[y1_train.index]
)

# Quantity-only model
quantity_model = tf.keras.Sequential([
    tf.keras.Input(shape=(2,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # quantity classes: 1–5
])
quantity_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fit with class weights
quantity_model.fit(
    X_train,
    y_quantity_class.iloc[y1_train.index],
    sample_weight=sample_weights_quantity,
    epochs=20,
    batch_size=32
)


Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 882us/step - accuracy: 0.1909 - loss: 1.7656 
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.2133 - loss: 1.6038
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.1286 - loss: 1.6345
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.1837 - loss: 1.5983
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step - accuracy: 0.1867 - loss: 1.6126
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - accuracy: 0.2621 - loss: 1.5710
Epoch 7/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.1897 - loss: 1.6090
Epoch 8/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 0.1960 - loss: 1.5794
Epoch 9/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - accuracy: 0.1160 - loss: 1.6344
Epoch 10/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.2180 - loss: 1.5661
Epoch 11/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - accuracy: 0.1575 - loss: 1.6196   
Epoch 12/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/

In [19]:
# Product + Area model
inputs = tf.keras.Input(shape=(2,))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(128, activation='relu')(x)

# Outputs
product_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='product')(x)
area_output = tf.keras.layers.Dense(len(area_encoder.classes_), activation='softmax', name='area')(x)

multi_model = tf.keras.Model(inputs=inputs, outputs=[product_output, area_output])
multi_model.compile(
    optimizer='adam',
    loss={
        'product': 'sparse_categorical_crossentropy',
        'area': 'sparse_categorical_crossentropy'
    },
    metrics={
        'product': 'accuracy',
        'area': 'accuracy'
    }
)

# Train (no sample weights)
multi_model.fit(
    X_train,
    {
        'product': y1_train,
        'area': y3_train
    },
    epochs=20,
    batch_size=32
)


Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 892us/step - area_accuracy: 0.0977 - area_loss: 2.2719 - loss: 5.7625 - product_accuracy: 0.0381 - product_loss: 3.4906 
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - area_accuracy: 0.1313 - area_loss: 2.2039 - loss: 5.4361 - product_accuracy: 0.0391 - product_loss: 3.2322   
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - area_accuracy: 0.1386 - area_loss: 2.2066 - loss: 5.4390 - product_accuracy: 0.0381 - product_loss: 3.2323   
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - area_accuracy: 0.1246 - area_loss: 2.2002 - loss: 5.4181 - product_accuracy: 0.0570 - product_loss: 3.2179
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - area_accuracy: 0.1301 - area_loss: 2.1998 - loss: 5.4141 - product_accuracy: 0.0625 - product_loss: 3.2144  
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - area_accuracy: 0.1254 - area_loss: 2.1994 - loss: 5.4061 - product_accuracy: 0.0520 - product_loss: 3.2067
Epoch 7/20
65/65 ━━━━━━━━━━

Prediction Function

In [20]:
def predict_top5_with_quantity_and_area(hour, day_name):
    encoded_day = day_encoder.transform([day_name])[0]
    input_data = np.array([[hour, encoded_day]])

    # Product Prediction (LightGBM)
    product_probs = lgbm_clf.predict_proba(input_data)[0]
    top5_indices = product_probs.argsort()[-5:][::-1]
    top5_product_ids = product_encoder.inverse_transform(top5_indices)
    top5_confidences = product_probs[top5_indices]

    # Quantity Prediction (Keras model)
    quantity_probs = quantity_model.predict(input_data, verbose=0)
    quantity = np.argmax(quantity_probs[0]) + 1

    # Area Prediction (from multi-output model)
    _, area_probs = multi_model.predict(input_data, verbose=0)
    area_idx = np.argmax(area_probs[0])
    predicted_area = area_encoder.inverse_transform([area_idx])[0]

    # Output
    return [
        {
            'product_id': pid,
            'confidence': round(float(conf), 4),
            'predicted_quantity': quantity,
            'predicted_area': predicted_area
        }
        for pid, conf in zip(top5_product_ids, top5_confidences)
    ]


Making Predictions

In [ ]:
results = predict_top5_with_quantity_and_area(19, 'Monday')
for i, r in enumerate(results, 1):
    print(f"{i}. Product ID: {r['product_id']} | Confidence: {r['confidence']:.4f} | Quantity: {r['predicted_quantity']} | Area: {r['predicted_area']}")

1. Product ID: 542696 | Confidence: 0.1830 | Quantity: 2 | Area: SW Sacramento
2. Product ID: 644730 | Confidence: 0.1764 | Quantity: 2 | Area: SW Sacramento
3. Product ID: 357966 | Confidence: 0.1031 | Quantity: 2 | Area: SW Sacramento
4. Product ID: 782729 | Confidence: 0.1018 | Quantity: 2 | Area: SW Sacramento
5. Product ID: 417009 | Confidence: 0.0933 | Quantity: 2 | Area: SW Sacramento


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
